# Notebook Setup: Importing Libraries

In [36]:
#  Face Mask Detection, Classification, and Segmentation Project

# Import basic libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import zipfile

# For handcrafted feature extraction (HOG)
from skimage.feature import hog

# For ML classifiers and evaluation
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

# For CNN using deep learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Question A.i: Extract Handcrafted Features from the Dataset

Task: Load the dataset and extract handcrafted features (using HOG) from each image.

In [8]:
import zipfile
import cv2
import numpy as np
from skimage.feature import hog

# Function to print the structure of the zip file (for debugging)
def print_zip_structure(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        print("Files in zip:", file_list)

# Set the zip file path
zip_file_path = '/content/finaldataset.zip'

# Print the structure to verify the folder names (optional)
print_zip_structure(zip_file_path)

# Function to load image paths and their corresponding labels from the zip file
def load_dataset(zip_path, base_folder='finaldataset/'):
    """
    base_folder: The top-level folder in the zip file containing the image subfolders.
    """
    categories = ['with_mask', 'without_mask']
    data = []
    labels = []

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        for label, category in enumerate(categories):
            # Build the expected prefix, e.g. "finaldataset/with_mask/"
            prefix = base_folder + category + '/'
            for img_name in file_list:
                # Check if the file name starts with the proper prefix and ensure it's not a directory
                if img_name.startswith(prefix) and not img_name.endswith('/'):
                    data.append(img_name)
                    labels.append(label)
    return data, labels

# Load image paths and labels (set base_folder according to your structure)
image_paths, labels = load_dataset(zip_file_path, base_folder='finaldataset/')
print("Total images loaded:", len(image_paths))

# Function to extract HOG features from an image in the zip file
def extract_hog_features(image_path, image_size=(64, 64), zip_path=zip_file_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        try:
            with zip_ref.open(image_path, 'r') as image_file:
                img_bytes = image_file.read()
                img_array = np.frombuffer(img_bytes, np.uint8)
                img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        except Exception as e:
            print(f"Error reading {image_path}: {e}")
            return None

        if img is None:
            return None
        img = cv2.resize(img, image_size)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features, _ = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                          cells_per_block=(2, 2), block_norm='L2-Hys',
                          visualize=True, transform_sqrt=True)
        return features

# Loop over images to extract HOG features and collect valid labels
features_list = []
valid_labels = []
for path, label in zip(image_paths, labels):
    feat = extract_hog_features(path)
    if feat is not None:
        features_list.append(feat)
        valid_labels.append(label)

features_array = np.array(features_list)
labels_array = np.array(valid_labels)
print("Extracted feature array shape:", features_array.shape)

Files in zip: ['finaldataset/', 'finaldataset/.DS_Store', '__MACOSX/finaldataset/._.DS_Store', 'finaldataset/with_mask/', '__MACOSX/finaldataset/._with_mask', 'finaldataset/without_mask/', '__MACOSX/finaldataset/._without_mask', 'finaldataset/with_mask/1361b.jpg', '__MACOSX/finaldataset/with_mask/._1361b.jpg', 'finaldataset/with_mask/354b.jpg', '__MACOSX/finaldataset/with_mask/._354b.jpg', 'finaldataset/with_mask/1731b.jpg', '__MACOSX/finaldataset/with_mask/._1731b.jpg', 'finaldataset/with_mask/704b.jpg', '__MACOSX/finaldataset/with_mask/._704b.jpg', 'finaldataset/with_mask/2058b.jpg', '__MACOSX/finaldataset/with_mask/._2058b.jpg', 'finaldataset/with_mask/211b.jpg', '__MACOSX/finaldataset/with_mask/._211b.jpg', 'finaldataset/with_mask/641b.jpg', '__MACOSX/finaldataset/with_mask/._641b.jpg', 'finaldataset/with_mask/1674b.jpg', '__MACOSX/finaldataset/with_mask/._1674b.jpg', 'finaldataset/with_mask/883b.png', '__MACOSX/finaldataset/with_mask/._883b.png', 'finaldataset/with_mask/1766b.png'

A HOG feature vector of length 1764 means that each image is represented by 1764 numbers that capture its gradient orientations and edge information.

We extracted SIFT features also and decided that HOG is better based on the performance.

In [65]:
import zipfile
import cv2
import numpy as np

# Function to extract SIFT features from an image stored in the zip file.
def extract_sift_features(image_path, image_size=(64, 64), zip_path='/content/finaldataset.zip'):
    # Open the image from the zip file.
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        with zip_ref.open(image_path, 'r') as image_file:
            img_bytes = image_file.read()
            img_array = np.frombuffer(img_bytes, np.uint8)
            img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    if img is None:
        return None
    # Resize and convert to grayscale.
    img = cv2.resize(img, image_size)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Create a SIFT object.
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)

    # If no descriptors are found, return a zero vector.
    if descriptors is None or len(descriptors) == 0:
        return np.zeros(128)

    # Pool the descriptors by taking their average to produce a fixed-length vector.
    feature_vector = descriptors.mean(axis=0)
    return feature_vector

# Loop over images (assuming you have lists 'image_paths' and 'labels' as defined in A(i))
sift_features_list = []
valid_labels_sift = []
for path, label in zip(image_paths, labels):
    feat = extract_sift_features(path)
    if feat is not None:
        sift_features_list.append(feat)
        valid_labels_sift.append(label)

sift_features_array = np.array(sift_features_list)
labels_array_sift = np.array(valid_labels_sift)
print("Extracted SIFT feature array shape:", sift_features_array.shape)

Extracted SIFT feature array shape: (4095, 128)


# Question A.ii: Train and Evaluate Two Machine Learning Classifiers(we used three)

Task: Using the extracted HOG features, train at least two classifiers (SVM and MLP) for binary classification (“with mask” vs. “without mask”).

Split the data into train and test

In [12]:
# Split the features and labels into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features_array, labels_array,
                                                    test_size=0.2, random_state=42)


SVM Classifier:

In [13]:
# ----- Train SVM Classifier -----
# linear kernel: initial
svm_classifier = svm.SVC(kernel='rbf', probability=True, random_state=42)
svm_classifier.fit(X_train, y_train)
y_pred_svm = svm_classifier.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy:", accuracy_svm)
#print("SVM Classification Report:\n", classification_report(y_test, y_pred_svm))

SVM Accuracy: 0.9352869352869353


Neural Network:

In [26]:
def create_custom_nn(input_dim):
    model = Sequential()
    # First hidden layer
    model.add(Dense(256, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.5))
    # Second hidden layer
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    # Third hidden layer (newly added)
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    # Output layer for binary classification
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create and train the updated custom neural network
nn_model = create_custom_nn(input_dim=X_train.shape[1])
nn_model.summary()

# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = nn_model.fit(
    X_train, y_train,
    epochs=50,          # You can adjust the number of epochs as needed
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate the updated custom neural network
loss_nn, accuracy_nn = nn_model.evaluate(X_test, y_test)
print("Neural Network Accuracy:", accuracy_nn)

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_34 (Dense)                     │ (None, 256)                 │         451,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_25 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_36 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 493,057 (1.88 MB)

 Trainable params: 493,057 (1.88 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5821 - loss: 0.6826 - val_accuracy: 0.8323 - val_loss: 0.4301
Epoch 2/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7768 - loss: 0.4777 - val_accuracy: 0.8476 - val_loss: 0.3288
Epoch 3/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.8544 - loss: 0.3487 - val_accuracy: 0.8735 - val_loss: 0.2851
Epoch 4/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.8711 - loss: 0.3163 - val_accuracy: 0.8796 - val_loss: 0.2590
Epoch 5/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8661 - loss: 0.3204 - val_accuracy: 0.8735 - val_loss: 0.2720
Epoch 6/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8747 - loss: 0.2912 - val_accuracy: 0.8887 - val_loss: 0.2420
Epoch 7/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8903 - loss: 0.2566 - val_accuracy: 0.8704 - val_loss: 0.2556
Epoch 8/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8948 - loss: 0.2515 - val_accuracy: 0.8720 - v

We added dropout layers, used adam optimiser, experimented with number of hidden layers and number of nodes in each layer to get the best result possible

XGBoost:

In [40]:
'''# Create and train the XGBoost classifier
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# Predict and evaluate
y_pred_xgb = xgb_model.predict(X_test)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("XGBoost Accuracy:", accuracy_xgb)'''

# Create DMatrices from training and testing data
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define a baseline parameter dictionary
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'error',        # error = 1 - accuracy
    'max_depth': 10,
    'eta': 0.1,                    # Learning rate
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0,
    'min_child_weight': 1,
    'n_jobs': -1,
    'seed': 42,
    'tree_method': 'hist'          # Use hist for faster training on CPU
}

# Use xgb.cv with early stopping to determine the best number of boosting rounds
cv_results = xgb.cv(
    params=params,
    dtrain=dtrain,
    num_boost_round=1000,
    nfold=5,
    metrics={'error'},
    early_stopping_rounds=10,
    verbose_eval=50  # Print progress every 50 rounds
)

best_num_rounds = len(cv_results)
print("Best number of rounds from CV:", best_num_rounds)

# Train the final model using the best number of boosting rounds from CV
final_model = xgb.train(params, dtrain, num_boost_round=best_num_rounds)

# Make predictions on the test set and evaluate accuracy
y_pred = final_model.predict(dtest)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred]
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy for tuned XGBoost: ", accuracy)

[0]	train-error:0.10897+0.00865	test-error:0.25488+0.00669
[50]	train-error:0.00000+0.00000	test-error:0.10317+0.00874
[95]	train-error:0.00000+0.00000	test-error:0.09584+0.01136
Best number of rounds from CV: 86
Test Accuracy for tuned XGBoost:  0.9242979242979243


So we xgb.cv with early stopping to determine the best number of boosting rounds to experiment with various hyperparameters and get the best results

# Question A.iii: Compare Classifier Accuracies

Summary of Results

In [42]:
# Compare the accuracies of SVM, Neural Network, and XGBoost

print("=== Comparison of Classifier Accuracies ===")
print("SVM Accuracy: {:.2f}%".format(accuracy_svm * 100))
print("Custom Neural Network Accuracy: {:.2f}%".format(accuracy_nn * 100))
print("XGBoost Accuracy: {:.2f}%".format(accuracy * 100))

=== Comparison of Classifier Accuracies ===
SVM Accuracy: 93.53%
Custom Neural Network Accuracy: 91.09%
XGBoost Accuracy: 92.43%


# Question B.i: Design and Train a CNN for Binary Classification

Task: Using the same dataset, design and train a Convolutional Neural Network (CNN) for binary classification.

In [46]:
import zipfile
import cv2
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# -------------------------
# Step 1: Load dataset information from the zip file
def load_dataset(zip_path, base_folder='finaldataset/'):
    """
    Scans the zip file and returns a list of image file paths (inside the zip)
    and their corresponding labels. Assumes two subfolders: 'with_mask' and 'without_mask'.
    """
    categories = ['with_mask', 'without_mask']
    data = []
    labels = []

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        for label, category in enumerate(categories):
            prefix = base_folder + category + '/'
            for img_name in file_list:
                if img_name.startswith(prefix) and not img_name.endswith('/'):
                    data.append(img_name)
                    labels.append(label)
    return data, labels

# Set the zip file path (update as needed)
zip_file_path = '/content/finaldataset.zip'
image_paths, labels = load_dataset(zip_file_path, base_folder='finaldataset/')
print("Total images loaded from zip:", len(image_paths))

# -------------------------
# Step 2: Split the data into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)
print("Training samples:", len(train_paths))
print("Validation samples:", len(val_paths))

# -------------------------
# Step 3: Define a custom data generator that reads images from the zip file
class ZipDataGenerator(Sequence):
    def __init__(self, zip_path, file_paths, labels, batch_size, img_size=(64, 64), shuffle=True):
        self.zip_path = zip_path
        self.file_paths = file_paths
        self.labels = np.array(labels)
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.file_paths))
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.file_paths) / self.batch_size)

    def __getitem__(self, index):
        batch_indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        batch_files = [self.file_paths[i] for i in batch_indices]
        batch_labels = self.labels[batch_indices]

        batch_images = []
        with zipfile.ZipFile(self.zip_path, 'r') as zf:
            for file in batch_files:
                try:
                    with zf.open(file) as f:
                        file_bytes = f.read()
                        img_array = np.frombuffer(file_bytes, np.uint8)
                        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
                except Exception as e:
                    print(f"Error reading {file}: {e}")
                    img = None
                if img is None:
                    # If image reading fails, use a blank image
                    img = np.zeros((self.img_size[0], self.img_size[1], 3), dtype=np.uint8)
                else:
                    img = cv2.resize(img, self.img_size)
                # Scale pixel values to [0, 1]
                img = img.astype(np.float32) / 255.0
                batch_images.append(img)
        return np.array(batch_images), np.array(batch_labels)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

# Define image dimensions and batch size
img_height, img_width = 64, 64
batch_size = 32

# Create training and validation generators using the zip file
train_gen = ZipDataGenerator(zip_file_path, train_paths, train_labels, batch_size, img_size=(img_height, img_width))
val_gen   = ZipDataGenerator(zip_file_path, val_paths, val_labels, batch_size, img_size=(img_height, img_width), shuffle=False)

# -------------------------
# Step 4: Define the CNN model architecture
def create_cnn_model():
    model = Sequential()
    # First Convolutional Block
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second Convolutional Block
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Third Convolutional Block
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    # Output layer for binary classification
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Create the CNN model and display its summary
cnn_model = create_cnn_model()
cnn_model.summary()

# -------------------------
# Step 5: Train the CNN model
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
epochs = 10  # Adjust epochs as needed

history = cnn_model.fit(
    train_gen,
    steps_per_epoch=len(train_gen),
    epochs=epochs,
    validation_data=val_gen,
    validation_steps=len(val_gen),
    callbacks=[early_stop],
    verbose=1
)

# -------------------------
# Step 6: Evaluate the model on the validation set
val_loss, val_accuracy = cnn_model.evaluate(val_gen, verbose=0)
print("Validation Loss: {:.4f}".format(val_loss))
print("Validation Accuracy: {:.2f}%".format(val_accuracy * 100))

Total images loaded from zip: 4095
Training samples: 3276
Validation samples: 819


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_27 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_39 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 683,329 (2.61 MB)

 Trainable params: 683,329 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 50s 455ms/step - accuracy: 0.6988 - loss: 0.5499 - val_accuracy: 0.9011 - val_loss: 0.2511
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 76s 401ms/step - accuracy: 0.9119 - loss: 0.2404 - val_accuracy: 0.9316 - val_loss: 0.1810
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 41s 396ms/step - accuracy: 0.9341 - loss: 0.1852 - val_accuracy: 0.9328 - val_loss: 0.1681
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 42s 402ms/step - accuracy: 0.9358 - loss: 0.1771 - val_accuracy: 0.9402 - val_loss: 0.1471
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 40s 391ms/step - accuracy: 0.9475 - loss: 0.1472 - val_accuracy: 0.9402 - val_loss: 0.1580
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 44s 423ms/step - accuracy: 0.9508 - loss: 0.1499 - val_accuracy: 0.9536 - val_loss: 0.1219
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 79s 394ms/step - accuracy: 0.9545 - loss: 0.1346 - val_accuracy: 0.9231 - val_loss: 0.1956
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 40s 389ms/step - accuracy: 0.9590 - loss: 0

# Question B.ii: Hyperparameter Variations for the CNN

Task: Experiment with variations (e.g., batch size, activation function, learning rate, optimizer) in the CNN design and train a variant model.

In [51]:
# Define a function to create the CNN model with variable hyperparameters
def create_cnn_model_variant(learning_rate=1e-3, optimizer_name='adam', final_activation='sigmoid', dropout_rate=0.5):
    model = Sequential()
    # First Convolutional Block
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second Convolutional Block
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Third Convolutional Block
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation=final_activation))

    # Select optimizer based on name
    if optimizer_name.lower() == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name.lower() == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer_name.lower() == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # default

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define a list of hyperparameter configurations to try
hyperparams = [
    {"learning_rate": 1e-2, "optimizer": "adam",    "batch_size": 16, "final_activation": "sigmoid", "dropout_rate": 0.3},
    {"learning_rate": 1e-3, "optimizer": "adam",    "batch_size": 32, "final_activation": "sigmoid", "dropout_rate": 0.5},
    {"learning_rate": 1e-3, "optimizer": "adam",    "batch_size": 64, "final_activation": "sigmoid", "dropout_rate": 0.5},
    {"learning_rate": 1e-4, "optimizer": "adam",    "batch_size": 64, "final_activation": "sigmoid", "dropout_rate": 0.5},
    {"learning_rate": 1e-4, "optimizer": "sgd",     "batch_size": 32, "final_activation": "sigmoid", "dropout_rate": 0.3},
    {"learning_rate": 1e-4, "optimizer": "rmsprop", "batch_size": 32, "final_activation": "sigmoid", "dropout_rate": 0.5},
    {"learning_rate": 1e-3, "optimizer": "adam",    "batch_size": 32, "final_activation": "relu",    "dropout_rate": 0.5},
    {"learning_rate": 1e-3, "optimizer": "adam",    "batch_size": 32, "final_activation": "softmax",    "dropout_rate": 0.5},
]

results = []
# Set number of epochs for the hyperparameter experiment (using a small number for quick evaluation)
exp_epochs = 5

# Loop over each hyperparameter configuration
for config in hyperparams:
    print("Training configuration:", config)

    # Reinitialize data generators with the specified batch size
    current_batch_size = config["batch_size"]
    train_gen_exp = ZipDataGenerator(zip_file_path, train_paths, train_labels, batch_size=current_batch_size, img_size=(img_height, img_width))
    val_gen_exp = ZipDataGenerator(zip_file_path, val_paths, val_labels, batch_size=current_batch_size, img_size=(img_height, img_width), shuffle=False)

    # Create the model with the current hyperparameters
    model_exp = create_cnn_model_variant(learning_rate=config["learning_rate"],
                                         optimizer_name=config["optimizer"],
                                         final_activation=config["final_activation"],
                                         dropout_rate=config["dropout_rate"])

    # Use early stopping to prevent overfitting
    early_stop_exp = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, verbose=0)

    # Train the model
    history_exp = model_exp.fit(
        train_gen_exp,
        steps_per_epoch=len(train_gen_exp),
        epochs=exp_epochs,
        validation_data=val_gen_exp,
        validation_steps=len(val_gen_exp),
        callbacks=[early_stop_exp],
        verbose=0
    )

    # Evaluate the model on the validation set
    val_loss_exp, val_acc_exp = model_exp.evaluate(val_gen_exp, verbose=0)
    print("Config:", config, "Validation Accuracy: {:.2f}%".format(val_acc_exp * 100))

    results.append({
        "config": config,
        "val_loss": val_loss_exp,
        "val_accuracy": val_acc_exp
    })
    print("------------------------------------------------")

# Summarize the results in a DataFrame
df_results = pd.DataFrame(results)
print("Hyperparameter Experiment Results:")
print(df_results)

Training configuration: {'learning_rate': 0.01, 'optimizer': 'adam', 'batch_size': 16, 'final_activation': 'sigmoid', 'dropout_rate': 0.3}
Config: {'learning_rate': 0.01, 'optimizer': 'adam', 'batch_size': 16, 'final_activation': 'sigmoid', 'dropout_rate': 0.3} Validation Accuracy: 92.06%
------------------------------------------------
Training configuration: {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 32, 'final_activation': 'sigmoid', 'dropout_rate': 0.5}
Config: {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 32, 'final_activation': 'sigmoid', 'dropout_rate': 0.5} Validation Accuracy: 94.38%
------------------------------------------------
Training configuration: {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 64, 'final_activation': 'sigmoid', 'dropout_rate': 0.5}
Config: {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 64, 'final_activation': 'sigmoid', 'dropout_rate': 0.5} Validation Accuracy: 93.77%
----------------------------

/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


Config: {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 32, 'final_activation': 'softmax', 'dropout_rate': 0.5} Validation Accuracy: 44.69%
------------------------------------------------
Hyperparameter Experiment Results:
                                              config  val_loss  val_accuracy
0  {'learning_rate': 0.01, 'optimizer': 'adam', '...  0.229737      0.920635
1  {'learning_rate': 0.001, 'optimizer': 'adam', ...  0.140560      0.943834
2  {'learning_rate': 0.001, 'optimizer': 'adam', ...  0.144613      0.937729
3  {'learning_rate': 0.0001, 'optimizer': 'adam',...  0.252372      0.904762
4  {'learning_rate': 0.0001, 'optimizer': 'sgd', ...  0.687361      0.543346
5  {'learning_rate': 0.0001, 'optimizer': 'rmspro...  0.275601      0.886447
6  {'learning_rate': 0.001, 'optimizer': 'adam', ...  0.528763      0.727717
7  {'learning_rate': 0.001, 'optimizer': 'adam', ...  0.132284      0.446886


In [55]:
# Define the new hyperparameter configurations to try
new_hyperparams = [
    {"learning_rate": 1e-3, "optimizer": "adam",    "batch_size": 32, "final_activation": "sigmoid", "dropout_rate": 0.3},
    {"learning_rate": 1e-3, "optimizer": "adam", "batch_size": 16, "final_activation": "sigmoid", "dropout_rate": 0.4}
]

new_results = []
exp_epochs = 5  # Use a reduced number of epochs for quick evaluation

# Loop over each new hyperparameter configuration
for config in new_hyperparams:
    print("Training with new configuration:", config)

    # Reinitialize data generators with the new batch size
    train_gen_new = ZipDataGenerator(zip_file_path, train_paths, train_labels,
                                     batch_size=config["batch_size"], img_size=(img_height, img_width))
    val_gen_new = ZipDataGenerator(zip_file_path, val_paths, val_labels,
                                   batch_size=config["batch_size"], img_size=(img_height, img_width), shuffle=False)

    # Create the model with the current hyperparameters
    model_new = create_cnn_model_variant(
        learning_rate=config["learning_rate"],
        optimizer_name=config["optimizer"],
        final_activation=config["final_activation"],
        dropout_rate=config["dropout_rate"]
    )

    # Use early stopping to prevent overfitting
    early_stop_new = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, verbose=0)

    # Train the model on the new configuration
    history_new = model_new.fit(
        train_gen_new,
        steps_per_epoch=len(train_gen_new),
        epochs=exp_epochs,
        validation_data=val_gen_new,
        validation_steps=len(val_gen_new),
        callbacks=[early_stop_new],
        verbose=1
    )

    # Evaluate the model on the validation set
    val_loss_new, val_acc_new = model_new.evaluate(val_gen_new, verbose=0)
    print("New Config:", config, "Validation Accuracy: {:.2f}%".format(val_acc_new * 100))

    # Append the results
    new_results.append({
        "config": config,
        "val_loss": val_loss_new,
        "val_accuracy": val_acc_new
    })
    print("------------------------------------------------")

# Create a DataFrame from the new results and append to the existing df_results
new_df = pd.DataFrame(new_results)
df_results = pd.concat([df_results, new_df], ignore_index=True)

print("Updated Hyperparameter Experiment Results:")
print(df_results)

Training with new configuration: {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 32, 'final_activation': 'sigmoid', 'dropout_rate': 0.3}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
103/103 ━━━━━━━━━━━━━━━━━━━━ 44s 407ms/step - accuracy: 0.7986 - loss: 0.4282 - val_accuracy: 0.8803 - val_loss: 0.3173
Epoch 2/5
103/103 ━━━━━━━━━━━━━━━━━━━━ 80s 391ms/step - accuracy: 0.9089 - loss: 0.2429 - val_accuracy: 0.9231 - val_loss: 0.2009
Epoch 3/5
103/103 ━━━━━━━━━━━━━━━━━━━━ 40s 386ms/step - accuracy: 0.9180 - loss: 0.2241 - val_accuracy: 0.9463 - val_loss: 0.1399
Epoch 4/5
103/103 ━━━━━━━━━━━━━━━━━━━━ 40s 389ms/step - accuracy: 0.9568 - loss: 0.1246 - val_accuracy: 0.9402 - val_loss: 0.1380
Epoch 5/5
103/103 ━━━━━━━━━━━━━━━━━━━━ 42s 396ms/step - accuracy: 0.9503 - loss: 0.1328 - val_accuracy: 0.9585 - val_loss: 0.1154
New Config: {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 32, 'final_activation': 'sigmoid', 'dropout_rate': 0.3} Validation Accuracy: 95.85%
------------------------------------------------
Training with new configuration: {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 16, 'final_activation': 'sigmoid', 'dropout_rate': 0

In [60]:
# Ensure that all column contents are fully visible
pd.set_option('display.max_colwidth', None)

# Display the full DataFrame
print("Full df_results:")
print(df_results)

# Find the row with the best validation accuracy
best_row = df_results.loc[df_results['val_accuracy'].idxmax()]

print("\nBest Hyperparameter Configuration:")
print(best_row)

Full df_results:
                                                                                                                     config  \
0        {'learning_rate': 0.01, 'optimizer': 'adam', 'batch_size': 16, 'final_activation': 'sigmoid', 'dropout_rate': 0.3}   
1       {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 32, 'final_activation': 'sigmoid', 'dropout_rate': 0.5}   
2       {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 64, 'final_activation': 'sigmoid', 'dropout_rate': 0.5}   
3      {'learning_rate': 0.0001, 'optimizer': 'adam', 'batch_size': 64, 'final_activation': 'sigmoid', 'dropout_rate': 0.5}   
4       {'learning_rate': 0.0001, 'optimizer': 'sgd', 'batch_size': 32, 'final_activation': 'sigmoid', 'dropout_rate': 0.3}   
5   {'learning_rate': 0.0001, 'optimizer': 'rmsprop', 'batch_size': 32, 'final_activation': 'sigmoid', 'dropout_rate': 0.5}   
6          {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 32, 'final_activation':

In [62]:
# Save the full results DataFrame to a CSV file
df_results.to_csv("all_cnn_hyperparameters.csv", index=False)
print("All results saved to all_cnn_hyperparameters.csv")

# Find the best row again (if not already stored)
best_row = df_results.loc[df_results['val_accuracy'].idxmax()]
# Convert the best row into a DataFrame (so it saves as a single row)
best_df = pd.DataFrame([best_row])
best_df.to_csv("best_cnn_hyperparameters.csv", index=False)
print("Best result saved to best_cnn_hyperparameters.csv")

All results saved to all_cnn_hyperparameters.csv
Best result saved to best_cnn_hyperparameters.csv


Hence, we get the best accuracy of 95.84% using CNN

# Question B.iii: Compare the CNN’s Performance with the ML Classifiers

Task: Evaluate the CNN models on the validation set and compare their performance with the earlier ML classifiers.

In [64]:
# Set display option so the full content is shown (especially for the "config" column)
pd.set_option('display.max_colwidth', None)

# 1. Display the full CNN hyperparameter experiment results
print("Full CNN Hyperparameter Experiment Results:")
print(df_results)

# 2. Find the best CNN hyperparameter configuration (highest validation accuracy)
best_row = df_results.loc[df_results['val_accuracy'].idxmax()]
print("\nBest CNN Hyperparameter Configuration:")
print(best_row)

# 3. Compare the best CNN accuracy with ML classifier accuracies
# (Ensure that accuracy_svm, accuracy_nn, accuracy_xgb are defined from earlier experiments)
print("\n=== Classifier Comparison ===")
print("SVM Accuracy: {:.2f}%".format(accuracy_svm * 100))
print("Neural Network (Handcrafted Features) Accuracy: {:.2f}%".format(accuracy_nn * 100))
print("Tuned XGBoost Accuracy: {:.2f}%".format(accuracy * 100))
print("Best CNN Accuracy: {:.2f}%".format(best_row['val_accuracy'] * 100))

Full CNN Hyperparameter Experiment Results:
                                                                                                                     config  \
0        {'learning_rate': 0.01, 'optimizer': 'adam', 'batch_size': 16, 'final_activation': 'sigmoid', 'dropout_rate': 0.3}   
1       {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 32, 'final_activation': 'sigmoid', 'dropout_rate': 0.5}   
2       {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_size': 64, 'final_activation': 'sigmoid', 'dropout_rate': 0.5}   
3      {'learning_rate': 0.0001, 'optimizer': 'adam', 'batch_size': 64, 'final_activation': 'sigmoid', 'dropout_rate': 0.5}   
4       {'learning_rate': 0.0001, 'optimizer': 'sgd', 'batch_size': 32, 'final_activation': 'sigmoid', 'dropout_rate': 0.3}   
5   {'learning_rate': 0.0001, 'optimizer': 'rmsprop', 'batch_size': 32, 'final_activation': 'sigmoid', 'dropout_rate': 0.5}   
6          {'learning_rate': 0.001, 'optimizer': 'adam', 'batch_siz

Thus, to conclude, we see that obviously the CNN performs the best when compared with other ML classifiers as expected as traditional ML classifiers rely on handcrafted features (like HOG, SIFT), which might not capture all the nuances of the images. CNNs learn hierarchical features that can capture more complex patterns, potentially leading to higher accuracy.